In [14]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.io import imread
import os
import sys


# Current folder as project_root
# Going three layers of folders upwards to src
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
src_dir      = os.path.join(project_root)

# Add the src directory to the Python module search path
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

# Import functions from src
import importlib
import src.Dice_Score
importlib.reload(src.Dice_Score)

from src.pre_processing import gammacorrection
from src.Complete_Otsu_Global import otsu_threshold_skimage_like
from src.Dice_Score import dice_score
from src.Otsu_Local import local_otsu_fast



# Process single image and its groundtruth
def process_single(img_path: Path, gt_path: Path, g=None) -> float:
    """ 
    Reads one image and corresponding groundtruth, proccesses, segments and computes dice score for one image.
    """
    # Reads image and reads, binarizes groundtruth
    img = imread(img_path, as_gray=True)
    gt = imread(gt_path, as_gray=True)        

    # Gamma transformation
    img_gamma = gammacorrection(img, gamma=g)
    
    # 1nd Otsu
    binary1 = otsu_threshold_skimage_like(img_gamma)

    # 5. Compute Dice score
    return dice_score(binary1.flatten(), gt.flatten())

# -------------------------------------------------------------------
# Mainroutine: Going through all image-gt pairs and collect all dice scores of data set: NIH3T3
# -------------------------------------------------------------------

# Define gamma values to test (e.g., from 0.1 to 2.0 in steps of 0.1)
gamma_values = np.arange(0.1, 2.1, 0.1)

# Set paths
img_dir = Path(os.path.join(project_root, "data-git", "NIH3T3", "img"))
gt_dir = Path(os.path.join(project_root, "data-git", "NIH3T3", "gt"))

# Loop over gamma values
dice_score_means_NIH3T3 = []

for gamma in gamma_values:
    dice_scores = []

    for img_file in sorted(img_dir.glob("dna-*.png")):
        idx = img_file.stem.split('-')[-1]
        gt_file = gt_dir / f"{idx}.png"

        # Compute Dice score for current image and gamma
        score = process_single(img_file, gt_file, g=gamma)
        dice_scores.append(score)

    # Convert to numpy array and print mean
    dice_scores = np.array(dice_scores)
    dice_score_mean = np.mean(dice_scores)
    print(f"Gamma {gamma:.2f} → Mean Dice: {dice_scores.mean():.4f}")
    dice_score_means_NIH3T3.append(dice_score_mean)

np.save('Dice_scores/NIH3T3_dice_means_gamma', dice_score_means_NIH3T3)
    
# NEXT

# Set paths
# Set paths
img_dir = Path(os.path.join(project_root, "data-git", "N2DL-HeLa", "img"))
gt_dir = Path(os.path.join(project_root, "data-git", "N2DL-HeLa", "gt"))

# Loop over gamma values
dice_score_means_N2DL_HeLa = []

for gamma in gamma_values:
    dice_scores = []

    for img_file in sorted(img_dir.glob("t-*.tif")):
        idx = img_file.stem.split('-')[-1]
        gt_file = gt_dir / f"man_seg{idx}.tif"

        # Compute Dice score for current image and gamma
        score = process_single(img_file, gt_file, g=gamma)
        dice_scores.append(score)

    # Convert to numpy array and print mean
    dice_scores = np.array(dice_scores)
    dice_score_mean = np.mean(dice_scores)
    print(f"Gamma {gamma:.2f} → Mean Dice: {dice_scores.mean():.4f}")
    dice_score_means_N2DL_HeLa.append(dice_score_mean)

np.save('Dice_scores/N2DL-HeLa_dice_means_gamma', dice_score_means_N2DL_HeLa)


# NEXT

# Set paths
img_dir = Path(os.path.join(project_root, "data-git", "N2DH-GOWT1", "img"))
gt_dir = Path(os.path.join(project_root, "data-git", "N2DH-GOWT1", "gt"))

# Loop over gamma values
dice_score_means_N2DH_GOWT1 = []

for gamma in gamma_values:
    dice_scores = []

    for img_file in sorted(img_dir.glob("t-*.tif")):
        idx = img_file.stem.split('-')[-1]
        gt_file = gt_dir / f"man_seg{idx}.tif"

        # Compute Dice score for current image and gamma
        score = process_single(img_file, gt_file, g=gamma)
        dice_scores.append(score)

    # Convert to numpy array and print mean
    dice_scores = np.array(dice_scores)
    dice_score_mean = np.mean(dice_scores)
    print(f"Gamma {gamma:.2f} → Mean Dice: {dice_scores.mean():.4f}")
    dice_score_means_N2DH_GOWT1.append(dice_score_mean)

np.save('Dice_scores/N2DH-GOWT1_dice_means_gamma', dice_score_means_N2DH_GOWT1)

ValueError: Prediction and ground truth images must have the same shape.

In [15]:
img = imread(Path(os.path.join(project_root, "data-git", "NIH3T3", "img", "dna-0.png")), as_gray=True)
gt = imread(Path(os.path.join(project_root, "data-git", "NIH3T3", "gt", "0.png")), as_gray=True)

img = gammacorrection(img)

t = local_otsu_fast(img)

binary = (img > t).astype(np.uint8)
          
print(dice_score(img.flatten(), binary.flatten()))

print(len(binary.flatten()))
print(len(gt.flatten()))

0.7124043311707419
1376256
1376256
